### Create a movie of ACCESS-OM2-01 sea surface velocities in a South Polar Stereographic projection

In [1]:
import xarray as xr # for working with labelled multi-dimensional arrays
import numpy as np # for numerical operations                                                           
from pathlib import Path # to check if path already exists

import matplotlib.pyplot as plt # for matlab-like plotting                                              
import cartopy.crs as ccrs # for maps     
import cosima_cookbook as cc # for loading in data
import matplotlib.path as mpath
import time
import matplotlib.patheffects as PathEffects
from matplotlib import ticker
import cftime
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib import gridspec
import os

import warnings; warnings.filterwarnings('ignore') # suppress warnings

from dask.distributed import Client
client = Client(n_workers=16)        
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 16
Total threads: 32,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39477,Workers: 16
Dashboard: /proxy/8787/status,Total threads: 32
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:36485,Total threads: 2
Dashboard: /proxy/45099/status,Memory: 15.70 GiB
Nanny: tcp://127.0.0.1:37553,


### Global options to change according to user

In [38]:
# change this path to the one where you'd like to save your output files
project = 'e14'
username = 'mv7494'
frame_rate = 30 # 30 fps is a good option
resolution = 1080
movie_name='test.mp4'

save = '/g/data/'+project+'/'+username+'/movie_frames/10m_speed_ACCESS-OM2-01/'; print(save)
Path(save).mkdir(parents=True, exist_ok=True) # create folder if it does not yet exist

/g/data/e14/mv7494/movie_frames/10m_speed_ACCESS-OM2-01/


### Using the cosima cookbook to load in daily sea surface velocity maps and save them as individual figures

In [39]:
%%time
session = cc.database.create_session()
def fancy_plot(ax):
    ax.gridlines(color='grey', linewidth=1, alpha=1, # dots as grid lines
                  xlocs=range(-180, 180, 60), # longitude grid lines
                  ylocs= np.linspace(-45, -90, num=4)) # latitude grid lines
    # ax.coastlines(); # add coast lines
    theta = np.linspace(0, 2*np.pi, 100); center, radius = [0.5, 0.5], .5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    
    # add labels manually
    xlab =    [   .99,      0,    1.01,    -.01,     .5,     .55,    .55,    .55] # x-position of labels
    ylab =    [   .75,    .75,     .24,     .24,   -.05,    .725,    .85,   .605] # y-position of labels
    txt_lab = ['60°E', '60°W', '120°E', '120°W', '180°',  '60°S', '45°S', '75°S'] # label text

    # loop through the 7 labels and surround with white space for higher visibility
    for l in range(len(txt_lab)):
        ax1.text(xlab[l], ylab[l], txt_lab[l], horizontalalignment='center', transform=ax1.transAxes, 
                fontsize=16).set_path_effects([PathEffects.withStroke(linewidth=2, foreground='w')]) 
        
# ----------------------------------------------------------------------------------------------------------- #
depth     = [    17]  # 48.98 m depth, subsurface to avoid flickering from high-frequency surface variability #
sel_lat   = [0, 940]  # 81.09°S - 29.15°S                                                                     #
# ----------------------------------------------------------------------------------------------------------- #
for t in range(10):#range(365): # loop through the time dimension, creating a frame for each daily output field
    filename = save + '10m_speed_frame_' + str('%03d' % (t,))+'.png' # name of frame to save as .png file

    for f in range(2): # loop through the two variables, u and v to calculate the speed (speed = u^2+v^2)
        variables = ['u', 'v']
        field = cc.querying.getvar(expt='01deg_jra55v140_iaf', variable=variables[f], 
                                        session=session, frequency='1 daily',
                                        attrs={'cell_methods': 'time: mean'},
                                        start_time='2012-01-01 00:00:00', 
                                        end_time='2012-12-31 00:00:00', 
                                        chunks = {'yu_ocean': '200MB', 'xu_ocean': '200MB'})[t,depth[0],sel_lat[0]:sel_lat[1],:]
        if f == 0: u = field # zonal velocity
        if f == 1: v = field # meridional velocity
    speed = (u * u + v * v).load() # load 2D wind speed magnitude field into memory
    
    if os.path.isfile(filename) == True: # skip iteration if final .png file already exists
        print('Frame for '+str(speed.time)[36:46] + ' already done')
        continue   
        
    # initialise figure
    fig = plt.figure(figsize=(8,8),tight_layout=True,facecolor='w',edgecolor='k')
    gs = gridspec.GridSpec(1,1) 
    
    ax1 = plt.subplot(gs[0,0], projection=ccrs.SouthPolarStereo(central_longitude=0))
    ax1.set_extent([-180, 180, -90, -30], crs=ccrs.PlateCarree()) # extent of plot

    blue_marble = plt.imread('/g/data/ik11/grids/BlueMarble.tiff')
    blue_marble_extent = (-180, 180, -90, 90) # extent of the land surface figure (same as above)
    # Add pretty land using the COSIMA recipe: https://cosima-recipes.readthedocs.io/en/latest/DocumentedExamples/Bathymetry.html

    # ---------------------------------------------------------------------------------------------------------------- #
    ax1.imshow(blue_marble, extent=blue_marble_extent, transform=ccrs.PlateCarree(), origin='upper')
    p1 = speed.plot.contourf(ax=ax1,levels=np.linspace(-0,.8,21),cmap='Blues_r',
                             add_colorbar=False,extend='max',transform=ccrs.PlateCarree())
    # use function to add info (labels, land, etc), add title with date
    fancy_plot(ax1); plt.title('ACCESS-OM2-01, 50 m subsurface speed, '+str(speed.time)[36:46]+'\n', fontsize=16)
    # ---------------------------------------------------------------------------------------------------------------- #

    # add colour bar
    cax = fig.add_axes([.312, 0, .4, .012]) # position: [x0, y0, width, height] centered colour bar
    cb = plt.colorbar(p1, cax = cax, shrink=.5, orientation='horizontal') 
    cb.set_label(label='(m s$^{-1}$)', size=16) # colour bar label
    cb.ax.tick_params(labelsize=16); tick_locator = ticker.MaxNLocator(nbins=5) # five ticks
    cb.locator = tick_locator;cb.update_ticks()

    # --- saving as 300 dpi .PNG image in specified folder ----------------------------------------------- #
    plt.savefig(filename, dpi=300, facecolor='w', edgecolor='w', orientation='landscape', format=None,     #
                transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)                     #
    # --- end of script ---------------------------------------------------------------------------------- # 
    print('Frame for '+str(speed.time)[36:46] + ' done')
    if t != 1: plt.close(fig) # close figure if it's not the first one.
print('-------------------------') 
# Wall time: 2.33 s for just the frame of the figure
# Wall time: 1min 47s for one frame

Exception during reset or similar
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 558, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 22841001015104 and this is thread id 22838090901248.
Exception closing connection <sqlite3.Connection object at 0x14c573a98b70>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/sqlalchemy/pool/b

Frame for 2011-12-01 already done
Frame for 2011-12-02 already done
Frame for 2011-12-03 already done
Frame for 2011-12-04 already done
Frame for 2011-12-05 already done
Frame for 2011-12-06 already done
Frame for 2011-12-07 already done
Frame for 2011-12-08 already done
Frame for 2011-12-09 already done
Frame for 2011-12-10 already done
-------------------------
CPU times: user 15.4 s, sys: 2.43 s, total: 17.8 s
Wall time: 20.3 s


### Creating the animation from the individually saved figures
- Warning if file already exists

In [40]:
%%time
from subprocess import check_call
check_call('ffmpeg -framerate '+str(frame_rate)+
           ' -pattern_type glob -i "'+
           save+'/10m_speed_frame_*.png" -vf scale=-2:'+str(resolution)+',setsar=1 '+
           save+movie_name, shell=True)

ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10.3.0 (GCC)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1645746662877/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1645746662877/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-vaapi --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-libvpx --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1645746662877/_build_env/bin/pkg-config
  li

CPU times: user 116 ms, sys: 62.4 ms, total: 179 ms
Wall time: 1.01 s


frame=   10 fps=0.0 q=-1.0 Lsize=     110kB time=00:00:00.23 bitrate=3860.9kbits/s speed=0.309x    
video:109kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.894817%
[libx264 @ 0x55cdcb5a0840] frame I:1     Avg QP:23.37  size: 66885
[libx264 @ 0x55cdcb5a0840] frame P:4     Avg QP:26.97  size:  8733
[libx264 @ 0x55cdcb5a0840] frame B:5     Avg QP:31.23  size:  1827
[libx264 @ 0x55cdcb5a0840] consecutive B-frames: 20.0% 40.0%  0.0% 40.0%
[libx264 @ 0x55cdcb5a0840] mb I  I16..4:  5.3% 60.1% 34.6%
[libx264 @ 0x55cdcb5a0840] mb P  I16..4:  0.4%  0.6%  0.1%  P16..4:  7.8%  6.0%  4.7%  0.0%  0.0%    skip:80.4%
[libx264 @ 0x55cdcb5a0840] mb B  I16..4:  0.0%  0.2%  0.0%  B16..8:  7.4%  1.1%  0.4%  direct: 0.9%  skip:89.9%  L0:22.8% L1:56.3% BI:20.9%
[libx264 @ 0x55cdcb5a0840] 8x8 transform intra:60.1% inter:49.7%
[libx264 @ 0x55cdcb5a0840] coded y,u,v intra: 31.5% 18.2% 19.6% inter: 4.2% 0.6% 1.0%
[libx264 @ 0x55cdcb5a0840] i16 v,h,dc,p: 56% 26% 19%  0%
[libx264

0

### Play the video
- this works when using Google Chrome or Microsoft Edge
- this does not work when using Mozilla Firefox

In [41]:
%%time
from IPython.display import Video
Video(save + movie_name, embed=True, height=500)

CPU times: user 0 ns, sys: 1.27 ms, total: 1.27 ms
Wall time: 964 µs
